# 데이터 정리

In [ ]:
import boto3
s3 = boto3.resource('s3')
bucket = "sagemaker-jhgan-workspace"
my_bucket = s3.Bucket(bucket)

In [ ]:
for obj in my_bucket.objects.filter(Prefix="train/img"):
    print(obj.key)
    fname = obj.key.split("/")[1]
    copy_source = {
        "Bucket":bucket,
        "Key":obj.key
    }
    s3.meta.client.copy(copy_source, bucket, f"NII/{fname}")

In [ ]:
for obj in my_bucket.objects.filter(Prefix="test_noGT"):
    print(obj.key)
    fname = obj.key.split("/")[1]
    copy_source = {
        "Bucket":bucket,
        "Key":obj.key
    }
    s3.meta.client.copy(copy_source, bucket, f"NII/test_{fname}")

# 전처리

- 실행하기 전에 도커에 이미지 푸시해주자

```bash
aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 349048005035.dkr.ecr.us-east-2.amazonaws.com
docker tag <image name> <repository name>
docker push <repository name>
```


In [68]:
import boto3
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role
from sagemaker.session import Session
s3 = boto3.resource('s3')
bucket = "sagemaker-jhgan-workspace"
my_bucket = s3.Bucket(bucket)

In [69]:
inputObject = ProcessingInput(
        source=f's3://sagemaker-jhgan-workspace/sample',
        destination=f'/opt/ml/processing/input_data'
)

In [70]:
script_processor = Processor(
    image_uri = "349048005035.dkr.ecr.us-east-2.amazonaws.com/pggan:latest",
    role = get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.4xlarge',
    entrypoint = ["python", "/opt/ml/code/dataset_tool.py", "create_from_nii", '/opt/ml/processing/processed_data', '/opt/ml/processing/input_data'],
    sagemaker_session=Session(default_bucket = "sagemaker-jhgan-workspace")
)

In [72]:
script_processor.run(
    inputs=[inputObject],
    outputs=[ProcessingOutput(source='/opt/ml/processing/processed_data')],
    wait=True,
    job_name= "nii-to-tfrecords-3"
)


Job Name:  nii-to-tfrecords-3
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://sagemaker-jhgan-workspace/sample', 'LocalPath': '/opt/ml/processing/input_data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'S3Output': {'S3Uri': 's3://sagemaker-jhgan-workspace/nii-to-tfrecords-3/output/output-1', 'LocalPath': '/opt/ml/processing/processed_data', 'S3UploadMode': 'EndOfJob'}}]
....................Loading images from "/opt/ml/processing/input_data"
Creating dataset "/opt/ml/processing/processed_data"
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.p